In [5]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import matplotlib.pyplot as mlt
import seaborn as sp
from torch.autograd import Variable
from torch import autograd
from datetime import datetime
import matplotlib.pyplot as plt
import argparse
from datetime import timedelta
import torch.autograd.functional as F

In [4]:
def gen_batch_forecasting(batch_size, step, dset):
    real_dset = dset[step * batch_size: (step + 1) * batch_size]
    return real_dset

In [22]:
def train_mogrifier_lstm(model, optimizer_model, loss_function, real_train, real_test, step_per_epoch):
    
    errors_generator = []
    real_dataset = torch.empty((0, 1), dtype=torch.float32).to(device)
    gen_dataset = torch.empty((0, 1), dtype=torch.float32).to(device)
    epoch = 300
    
    for i in range(epoch):
        count = 0
        for step in range(step_per_epoch):
            # Data for training the discriminator
            real_data = gen_batch_forecasting(batch_size, count, real_train)
            real_label = gen_batch_forecasting(batch_size, count, real_test)

            model.zero_grad()
            generated_samples = model(real_data)

            L2 = torch.sum((torch.unsqueeze(real_label[:, 1], 1) - generated_samples)**2)
            g_loss = loss_function(generated_samples, torch.unsqueeze(real_label[:, 1], 1)) + L2
            g_loss.backward()
            optimizer_model.step()
            errors_generator.append(g_loss.item())
            
            count += 1
            if i == epoch - 1:
                real_dataset = torch.cat([real_dataset, torch.unsqueeze(real_label[:, 1], 1)], dim=0)
                gen_dataset = torch.cat([gen_dataset, generated_samples], dim=0)

            # Show loss
            if step == step_per_epoch - 1:
                print(f"Epoch: {i} Loss G.: {g_loss.item()}")

    return real_dataset, gen_dataset, errors_generator

In [21]:
def train_bi_lstm(model, optimizer_model, loss_function, real_train, real_test, step_per_epoch):
    
    errors_generator = []
    real_dataset = torch.empty((0, 1), dtype=torch.float32).to(device)
    gen_dataset = torch.empty((0, 1), dtype=torch.float32).to(device)
    epoch = 400
    
    for i in range(epoch):
        count = 0
        for step in range(step_per_epoch):
            # Data for training the discriminator
            real_data = gen_batch_forecasting(batch_size, count, real_train)
            real_label = gen_batch_forecasting(batch_size, count, real_test)

            model.zero_grad()
            generated_samples = model(real_data)

            L2 = torch.sum((torch.unsqueeze(real_label[:, 1], 1) - generated_samples)**2)
            g_loss = loss_function(generated_samples, torch.unsqueeze(real_label[:, 1], 1)) + L2
            g_loss.backward()
            optimizer_model.step()
            errors_generator.append(g_loss.item())
            
            count += 1
            if i == epoch - 1:
                real_dataset = torch.cat([real_dataset, torch.unsqueeze(real_label[:, 1], 1)], dim=0)
                gen_dataset = torch.cat([gen_dataset, generated_samples], dim=0)

            # Show loss
            if step == step_per_epoch - 1:
                print(f"Epoch: {i} Loss G.: {g_loss.item()}")

    return real_dataset, gen_dataset, errors_generator

In [23]:
def train_vanilla_lstm(model, optimizer_model, loss_function, real_train, real_test, step_per_epoch):
    
    errors_generator = []
    real_dataset = torch.empty((0, 1), dtype=torch.float32).to(device)
    gen_dataset = torch.empty((0, 1), dtype=torch.float32).to(device)
    epoch = 400
    
    for i in range(epoch):
        count = 0
        for step in range(step_per_epoch):
            # Data for training the discriminator
            real_data = gen_batch_forecasting(batch_size, count, real_train)
            real_label = gen_batch_forecasting(batch_size, count, real_test)

            model.zero_grad()
            generated_samples = model(real_data)

            L2 = torch.sum((torch.unsqueeze(real_label[:, 1], 1) - generated_samples)**2)
            g_loss = loss_function(generated_samples, torch.unsqueeze(real_label[:, 1], 1)) + L2
            g_loss.backward()
            optimizer_model.step()
            errors_generator.append(g_loss.item())
            
            count += 1
            if i == epoch - 1:
                real_dataset = torch.cat([real_dataset, torch.unsqueeze(real_label[:, 1], 1)], dim=0)
                gen_dataset = torch.cat([gen_dataset, generated_samples], dim=0)

            # Show loss
            if step == step_per_epoch - 1:
                print(f"Epoch: {i} Loss G.: {g_loss.item()}")

    return real_dataset, gen_dataset, errors_generator